In [49]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import pandas as pd
import numpy as np

plotly.offline.init_notebook_mode(connected=True)

In [8]:
df = pd.read_csv('cleaned_data.csv')
# df.shape
df.head(3)

,Unnamed: 0,City,State,Latitude,Longitude,Accuracy Score,Accuracy Type,State.1,County,Zip,Country,Congressional District,status,statusText,color
0,0,Aurora,Illinois,41.747452,-88.238622,1.0,place,IL,DuPage County,60504.0,US,IL11,0,Ordinance Active - With No Response,"rgb(255, 0, 0)"
1,1,Carbondale,Illinois,37.739004,-89.209109,1.0,place,IL,Jackson County,62901.0,US,IL12,0,Ordinance Active - With No Response,"rgb(255, 0, 0)"
2,2,Champaign,Illinois,40.107890,-88.244266,1.0,place,IL,Champaign County,61820.0,US,IL13,0,Ordinance Active - With No Response,"rgb(255, 0, 0)"


I'm going to update my values here. I won't persist these changes because a city's status may change multiple times and the size of the data set is very small, so this step takes very little time. 

In [18]:
updates_df = pd.read_csv('updates/updates_9_19_18.csv')
updates_df.head()

,City,State,MAP_STATUS,Unnamed: 3,Unnamed: 4
0,Aurora,Illinois,0,NaN,NaN
1,Carbondale,Illinois,0,NaN,Please put the color's number in the MAP_STATU...
2,Champaign,Illinois,0,NaN,NaN
3,Chicago,Illinois,0,NaN,Red = 0
4,Cicero,Illinois,0,NaN,Pink = 1


In [21]:
df['status'] = updates_df['MAP_STATUS']

df.loc[df['status'] == 4]

,Unnamed: 0,City,State,Latitude,Longitude,Accuracy Score,Accuracy Type,State.1,County,Zip,Country,Congressional District,status,statusText,color,text
12,12,Peoria,Illinois,40.693137,-89.589847,1.0,place,IL,Peoria County,61601.0,US,IL17,4,Ordinance Active - With No Response,"rgb(255, 0, 0)","Peoria, IL. Ordinance Active - With No Response"
63,63,Blue River,Colorado,40.087476,-108.804829,0.0,NaN,CO,NaN,NaN,NaN,CO3,4,Ordinance Active - With No Response,"rgb(255, 0, 0)","Blue River, CO. Ordinance Active - With No Res..."


So far so good

In [22]:
# Updating Ordinance Status

d_text = {0: 'Ordinance Active - With No Response', 
          1: 'Ordinance Active - With Response Indicating No Immediate Repeal',
          2: 'Ordinance Active - With Committment To Review', 
          3: 'Ordinance Halted - With Committment to Review',
          4: 'Ordinance Repealed'}

df['statusText'] = df['status'].map(d_text)

# Updating Marker Color 

d_color = {0: 'rgb(255, 0, 0)',
           1: 'rgb(255, 192, 203)',
           2: 'rgb(255, 165, 0)', 
           3: 'rgb(255, 255, 0)', 
           4: 'rgb(127, 255, 0)'}

df['color'] = df['status'].map(d_color)

Let's check to see if that works

In [23]:
df.loc[df['color'] == 'rgb(127, 255, 0)']

,Unnamed: 0,City,State,Latitude,Longitude,Accuracy Score,Accuracy Type,State.1,County,Zip,Country,Congressional District,status,statusText,color,text
12,12,Peoria,Illinois,40.693137,-89.589847,1.0,place,IL,Peoria County,61601.0,US,IL17,4,Ordinance Repealed,"rgb(127, 255, 0)","Peoria, IL. Ordinance Active - With No Response"
63,63,Blue River,Colorado,40.087476,-108.804829,0.0,NaN,CO,NaN,NaN,NaN,CO3,4,Ordinance Repealed,"rgb(127, 255, 0)","Blue River, CO. Ordinance Active - With No Res..."


Looks good.

In [28]:
df.loc[df['color'] == 'rgb(255, 192, 203)']

,Unnamed: 0,City,State,Latitude,Longitude,Accuracy Score,Accuracy Type,State.1,County,Zip,Country,Congressional District,status,statusText,color,text
15,15,Biloxi,Mississippi,30.409826,-88.880304,1.0,place,MS,Harrison County,39530.0,US,MS4,1,Ordinance Active - With Response Indicating No...,"rgb(255, 192, 203)","Biloxi, MS. Ordinance Active - With Response I..."


In [25]:
# This creates the text that appears at each point on the the scroll over
df['text'] = df['City'] + ', ' + df['State.1'] + '.' + ' ' + df['statusText']

In [26]:
# annnotation_text = "Note: This is not a complete map of communities with unconstitutional panhandling ordinances. Ordinances were targeted based on local partnerships and resources. More than 60% of communities surveyed in a 2016 survey had some form of panhandling ordinance."

In [82]:
#Building the actual map here 

data = [dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df['Longitude'],
        lat = df['Latitude'],
        text = df['text'],
        hoverinfo = 'text',
        mode = 'markers',
        marker = dict(
            size = 8,
            opacity = 0.85,
            symbol = 'circle',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'), 
            color = df['color'],
        ))]

layout = dict(
        title = 'Targeted Cities with Active Anti-Panhandling Laws',
        geo = dict(
            scope='usa',
            projection=dict(type='albers usa'),
            showland = True,
            landcolor = "rgb(180, 180, 180)",
            subunitcolor = "rgb(217, 217, 217)",
            countrycolor = "rgb(217, 217, 217)",
            countrywidth = 0.5,
            subunitwidth = 0.5,
        ),
    )

fig = dict(data = data, layout = layout)

# This code is used to create standalone HTML that is saved locally and opened in the browser.

plotly.offline.plot(fig, validate=False, filename='city_locations.html')

# This code is for use in displaying the map in Juypter notebook. 

# plotly.offline.iplot(fig, validate=False, filename='city_locations')

'file:///Users/flatironschool/Documents/Projects/NLCHP_Map/Repo/city_locations.html'

NOT YET OPERATIONAL, DO NOT TOUCH

In [81]:
# # by author
# traces_for_plot = []
# for key, value in author_num.items():
#     trace_df = doc_vectors_3d_with_author[doc_vectors_3d_with_author['author'] == value]
#     traces_for_plot.append(trace_df)
    
    

# traces_author = []
# for trace in traces_for_plot:
#     trace1 = go.Scatter3d(

                
#         x = trace['x'],
#         y = trace['y'],

#     mode='markers',
#     name = authors[trace['author'].values[0]],
#     marker=dict(
#         size=5,
#         color= 'rgba' + str(cmap_author[(trace['author'].values[0])]),
#     )
#     )
#     traces_author.append(trace1)
# authors_df = pd.DataFrame([train_corpus[i].tags[0] for i in range(0, len(train_corpus))])
# doc_vectors_3d_with_author['author'] = doc_vectors_3d_with_author['author'].map(author_num)

city_plots = pd.DataFrame(data = [df['Latitude'], df['Longitude'], df['text'], df['status'], df['statusText'], df['color']])
final_plots = city_plots.T
final_plots.columns = ['lat', 'lon', 'text', 'status', 'statusText', 'color']

traces_for_plot = []
for i in range(0,5): 
    trace_df = final_plots[final_plots['status'] == i]
    traces_for_plot.append(trace_df)
    
final_traces = []
for trace in traces_for_plot:
    final_trace = go.Scattergeo(
           locationmode = 'USA-states',
            lon = final_plots['lon'],
            lat = final_plots['lat'],
            text = final_plots['text'],
            hoverinfo = 'text',
            mode = 'markers',
            name = final_plots['statusText'].values[0],
            marker = dict(
                size = 8,
                opacity = 0.85,
                symbol = 'circle',
                line = dict(
                    width=1,
                    color='rgba(102, 102, 102)'), 
                color = final_plots['color']
            ))
    final_traces.append(final_trace)    
    
    
layout = go.Layout(showlegend=True, title = 'Targeted Cities with Active Anti-Panhandling Laws', 
                   geo = dict(scope = 'usa',
                            projection=dict(type='albers usa'),
                            showland = True,
                            landcolor = "rgb(180, 180, 180)",
                            subunitcolor = "rgb(217, 217, 217)",
                            countrycolor = "rgb(217, 217, 217)",
                            countrywidth = 0.5,
                            subunitwidth = 0.5))
                   
# # Make a figure object
fig = go.Figure(data=final_traces, layout=layout)

# Send to Plotly and show in notebook
iplot(fig, filename='test1')

In [78]:
print(len(traces_for_plot))
traces_for_plot[3]

4


,lat,lon,text,status,statusText,color
36,35.8495,-106.288,"Los Alamos, NM. Ordinance Halted - With Commit...",3,Ordinance Halted - With Committment to Review,"rgb(255, 255, 0)"
37,34.7658,-106.712,"Los Lunas, NM. Ordinance Halted - With Committ...",3,Ordinance Halted - With Committment to Review,"rgb(255, 255, 0)"
83,38.8772,-107.609,"Paonia, CO. Ordinance Halted - With Committmen...",3,Ordinance Halted - With Committment to Review,"rgb(255, 255, 0)"


In [36]:
city_plots

In [56]:
d_color[(final_plots['status'])]

TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [59]:
d_color[2]

'rgb(255, 165, 0)'

In [76]:
trace_df = final_plots[final_plots['status'] == 4]
trace_df

,lat,lon,text,status,statusText,color
12,40.6931,-89.5898,"Peoria, IL. Ordinance Repealed",4,Ordinance Repealed,"rgb(127, 255, 0)"
63,40.0875,-108.805,"Blue River, CO. Ordinance Repealed",4,Ordinance Repealed,"rgb(127, 255, 0)"


In [79]:
for i in range(0,4):
    print(i)

0
1
2
3


In [30]:
#Building the actual map here 

data = [dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df['Longitude'],
        lat = df['Latitude'],
        text = df['text'],
        hoverinfo = 'text',
        mode = 'markers',
        marker = dict(
            size = 8,
            opacity = 0.85,
            symbol = 'circle',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'), 
            color = df['color'],
        ))]

layout = dict(
        title = 'Targeted Cities with Active Anti-Panhandling Laws',
        geo = dict(
            scope='usa',
            projection=dict(type='albers usa'),
            showland = True,
            landcolor = "rgb(180, 180, 180)",
            subunitcolor = "rgb(217, 217, 217)",
            countrycolor = "rgb(217, 217, 217)",
            countrywidth = 0.5,
            subunitwidth = 0.5,
        ),
    )

fig = dict(data = data, layout = layout)

# This code is used to create standalone HTML that is saved locally and opened in the browser.

# plotly.offline.plot(fig, validate=False, filename='city_locations.html')

# This code is for use in displaying the map in Juypter notebook. 

plotly.offline.iplot(fig, validate=False, filename='city_locations')

In [80]:
# # by author
# traces_for_plot = []
# for key, value in author_num.items():
#     trace_df = doc_vectors_3d_with_author[doc_vectors_3d_with_author['author'] == value]
#     traces_for_plot.append(trace_df)
    
    

# traces_author = []
# for trace in traces_for_plot:
#     trace1 = go.Scatter3d(

                
#         x = trace['x'],
#         y = trace['y'],

#     mode='markers',
#     name = authors[trace['author'].values[0]],
#     marker=dict(
#         size=5,
#         color= 'rgba' + str(cmap_author[(trace['author'].values[0])]),
#     )
#     )
#     traces_author.append(trace1)
# authors_df = pd.DataFrame([train_corpus[i].tags[0] for i in range(0, len(train_corpus))])
# doc_vectors_3d_with_author['author'] = doc_vectors_3d_with_author['author'].map(author_num)

city_plots = pd.DataFrame(data = [df['Latitude'], df['Longitude'], df['text'], df['status'], df['statusText'], df['color']])
final_plots = city_plots.T
final_plots.columns = ['lat', 'lon', 'text', 'status', 'statusText', 'color']

traces_for_plot = []
for i in range(0,5): 
    trace_df = final_plots[final_plots['status'] == i]
    traces_for_plot.append(trace_df)
    
final_traces = []
for trace in traces_for_plot:
    final_trace = go.Scattergeo(
           locationmode = 'USA-states',
            lon = final_plots['lon'],
            lat = final_plots['lat'],
            text = final_plots['text'],
            hoverinfo = 'text',
            mode = 'markers',
            name = final_plots['statusText'].values[0],
            marker = dict(
                size = 8,
                opacity = 0.85,
                symbol = 'circle',
                line = dict(
                    width=1,
                    color='rgba(102, 102, 102)'), 
                color = final_plots['color']
            ))
    final_traces.append(final_trace)    
    
    
layout = go.Layout(showlegend=True, title = 'Targeted Cities with Active Anti-Panhandling Laws', 
                   geo = dict(scope = 'usa',
                            projection=dict(type='albers usa'),
                            showland = True,
                            landcolor = "rgb(180, 180, 180)",
                            subunitcolor = "rgb(217, 217, 217)",
                            countrycolor = "rgb(217, 217, 217)",
                            countrywidth = 0.5,
                            subunitwidth = 0.5))
                   
# # Make a figure object
fig = go.Figure(data=final_traces, layout=layout)

# Send to Plotly and show in notebook
iplot(fig, filename='test1')